In [1]:
from scipy.stats import multivariate_normal
from scipy.special import logsumexp
from glob import glob
import soundfile as sf
from os import path
import numpy as np
np.random.seed(seed=273)

In [17]:
class GaussianHMM(object):
    def __init__(self, n_states, n_dims):
        self.n_states = n_states
        self.n_dims = n_dims

    def init_gaussian_params(self, X):
        X_concat = np.concatenate(X)
        self.mu = np.zeros((self.n_states, self.n_dims))
        self.sigma = np.zeros((self.n_states, self.n_dims))
        for s in range(self.n_states):
            X_subset = X_concat[np.random.choice(len(X_concat), size=2, replace=False)]
            self.mu[s] = X_subset.mean(axis=0)
            self.sigma[s] = X_subset.var(axis=0)

    def init_hmm_params(self):
        self.pi = np.zeros(self.n_states)
        self.pi[0] = 1.
        self.A = np.zeros((self.n_states, self.n_states))
        for s in range(self.n_states - 1):
            self.A[s, s:s + 2] = .5
        self.A[-1, -1] = 1.

    def get_emissions(self, x):
        T, _ = x.shape
        log_B = np.zeros((self.n_states, T))
        for s in range(self.n_states):
            log_B[s] = multivariate_normal.logpdf(x, mean=self.mu[s], cov=np.diag(self.sigma[s]))
        return log_B

    def log_forward(self, log_pi, log_A, log_B):
        _, T = log_B.shape
        log_alpha = np.zeros(log_B.shape)
        for t in range(T):
            if t == 0:
                log_alpha[:, t] = log_pi + log_B[:, 0]
                #TODO: log alpha to time t
            else:
                log_alpha[:, t] = logsumexp(log_alpha[:, t - 1] + log_A, axis=1) + log_B[:, t]
                #TODO: log alpha to time t
        return log_alpha

    def log_backward(self, log_A, log_B):
        _, T = log_B.shape
        log_beta = np.zeros(log_B.shape)
        for t in range(T - 1, -1, -1):
            if t == T - 1:
                log_beta[:, t] = 0 # log(1) = 0
                #TODO: log beta from time t
            else:
                log_beta[:, t] = logsumexp(log_A + log_B[:, t + 1] + log_beta[:, t + 1], axis=1)
                #TODO: log beta from time t
        return log_beta
    
    def forward(self, B):
        _, T = B.shape
        alpha = np.empty((self.n_states, T))
        alpha[:, 0] = self.pi * B[:, 0]
        for t in range(1, T):
            alpha[:, t] = alpha[:, t - 1] @ self.A * B[:, t]
        return alpha
    
    def backward(self, B):
        _, T = B.shape
        beta = np.empty((self.n_states, T))
        beta[:, T - 1] = 1
        for t in range(T - 2, -1, -1):
            beta[:, t] = self.A * B[:, t + 1] @ beta[:, t + 1]
        return beta
    
    def score(self, x):
        T = len(x)
        log_B = self.get_emissions(x) # emission log probabilities
        B = np.exp(log_B)
        
        alpha = self.forward(B)
        beta = self.backward(B)
        prob = sum(alpha[:, -1])
        
        gamma = alpha * beta / prob
        xi = np.empty((T - 1, self.n_states, self.n_states))
        for t in range(T - 1):
            xi[t] = np.outer(alpha[:, t], beta[:, t + 1]) * self.A * B[:, t + 1]
        xi /= prob
        
        xi = xi.sum(axis=0) # sum over time
        xi /= xi.sum(axis=1, keepdims=True).clip(1e-1)
        
        return prob, alpha, beta, gamma, xi

    
    def log_score(self, x):
        T = len(x)

        log_pi = np.log(self.pi) # starting log probabilities
        log_A = np.log(self.A) # transition log probabilities
        log_B = self.get_emissions(x) # emission log probabilities
        
        # XXX: my forward algo needs log_A.T
        log_alpha = self.log_forward(log_pi, log_A.T, log_B)
        log_beta = self.log_backward(log_A, log_B)

        log_prob = logsumexp(log_alpha[:, -1])
        #TODO: log probability of observations
#         debug = logsumexp(log_pi + log_B[:, 0] + log_beta[:, 0])
#         assert np.isclose(log_prob, debug)

        gamma = np.exp(log_alpha + log_beta - log_prob)
        #TODO: posteriors

        xi = np.zeros((T - 1, self.n_states, self.n_states))
        for t in range(T - 1):
            xi[t] = log_alpha[:, t][:, None] + log_beta[:, t + 1] + log_A + log_B[:, t + 1]
            #TODO: transition prob i -> j for each t
        xi -= log_prob
        xi = np.exp(xi)
        
        xi = xi.sum(axis=0) # sum over time
        xi /= xi.sum(axis=1, keepdims=True).clip(1e-1) # normalize by state probabilities (sum transitions over j)

        return log_prob, log_alpha, log_beta, gamma, xi

In [18]:
def alpha_prob(alpha):
    return sum(alpha[:, -1])

def beta_prob(pi, B, beta):
    return sum(pi * B[:, 0] * beta[:, 0])

def log_alpha_prob(log_alpha):
    return logsumexp(log_alpha[:, -1])

def log_beta_prob(log_pi, log_B, log_beta):
    return logsumexp(log_pi + log_B[:, 0] + log_beta[:, 0])

In [23]:
X = np.random.random((4, 4))
gmm = GaussianHMM(3, 2)
gmm.init_gaussian_params(X)
gmm.init_hmm_params()
x = X[0][:, None]
T = len(x)
log_pi = np.log(gmm.pi) # starting log probabilities
log_A = np.log(gmm.A) # transition log probabilities
log_B = gmm.get_emissions(x) # emission log probabilities
B = np.exp(log_B)

<ipython-input-23-2198d3cbf15d>:7: RuntimeWarning: divide by zero encountered in log
  log_pi = np.log(gmm.pi) # starting log probabilities
<ipython-input-23-2198d3cbf15d>:8: RuntimeWarning: divide by zero encountered in log
  log_A = np.log(gmm.A) # transition log probabilities


In [24]:
# test score and log_score
log_prob, log_alpha, log_beta, gamma1, xi1 = gmm.log_score(x)
prob, alpha, beta, gamma2, xi2 = gmm.score(x)

<ipython-input-17-9e7a3fc5354a>:94: RuntimeWarning: divide by zero encountered in log
  log_pi = np.log(self.pi) # starting log probabilities
<ipython-input-17-9e7a3fc5354a>:95: RuntimeWarning: divide by zero encountered in log
  log_A = np.log(self.A) # transition log probabilities


In [25]:
log_prob, np.log(prob)

(-2.4682386643125236, -2.4682386643125236)

In [26]:
np.allclose(log_alpha, np.log(alpha)),\
np.allclose(log_beta, np.log(beta)),\
np.allclose(gamma1, gamma2)

<ipython-input-26-90d556559d8c>:1: RuntimeWarning: divide by zero encountered in log
  np.allclose(log_alpha, np.log(alpha)),\


(True, True, True)

In [27]:
np.allclose(xi1, xi2)

True

In [69]:
alpha = gmm.forward(B)
beta = gmm.backward(B)
alpha_prob(alpha), beta_prob(gmm.pi, B, beta)

(10.838380493498867, 10.838380493498867)

In [70]:
log_beta = gmm.log_backward(log_A, log_B)
assert np.allclose(log_beta, np.log(beta))
log_beta_prob(log_pi, log_B, log_beta),\
np.exp(log_beta_prob(log_pi, log_B, log_beta))

(2.3830935838813243, 10.838380493498864)

In [75]:
log_alpha = gmm.log_forward(log_pi, log_A.T, log_B)
assert np.allclose(log_alpha, np.log(alpha))
log_alpha_prob(log_alpha),\
np.exp(log_alpha_prob(log_alpha))

<ipython-input-75-aacc4f1e57aa>:2: RuntimeWarning: divide by zero encountered in log
  assert np.allclose(log_alpha, np.log(alpha))


(2.383093583881324, 10.83838049349886)

In [84]:
gamma = alpha * beta / alpha_prob(alpha)
log_gamma = log_alpha + log_beta - log_alpha_prob(log_alpha)
assert np.allclose(log_gamma, np.log(gamma))

<ipython-input-84-1c23e57ced54>:3: RuntimeWarning: divide by zero encountered in log
  assert np.allclose(log_gamma, np.log(gamma))
